# Data Scientist

<div class="alert alert-info">
Before running this notebook, start a PyGrid Domain. For a local instance using Docker you can use <code>scripts/start_grid_domain.sh</code>.
</div>

In [1]:
from math import prod
from typing import Any

import torch as T

import syft as sy
from syft import make_plan
from syft.federated.model_centric_fl_client import ModelCentricFLClient
from syft.lib.python.int import Int
from syft.lib.python.list import List

## Step 1: Specify configuration

In [2]:
NAME = "q-learning"
VERSION = "0.0.0"
GRID_ADDRESS = "localhost:5000"

CLIENT_CONFIG = {
    "name": NAME,
    "version": VERSION,
    "alpha": 0.1,
    "gamma": 1.0,
    "min_epsilon": 0.1,
    "epsilon_reduction": 0.001,
    "n_train_iterations": 1000,
    "n_test_iterations": 100,
    # "max_updates": 1,  # custom syft.js option that limits number of training loops per worker
}

SERVER_CONFIG = {
    "min_workers": 2,
    "max_workers": 2,
    "pool_selection": "random",
    "do_not_reuse_workers_until_cycle": 6,
    "cycle_length": 28800,  # max cycle length in seconds
    "num_cycles": 30,  # max number of cycles
    "max_diffs": 1,  # number of diffs to collect before avg
    "minimum_upload_speed": 0,
    "minimum_download_speed": 0,
    "iterative_plan": True,  # tells PyGrid that avg plan is executed per diff
}

CARTPOLE_DIMS = (1, 1, 12, 6, 2)

## Step 2: Define the model to be hosted

In [3]:
# Hacky way to make a Q-table which PyGrid will host.
# 1. We need the model itself to be a sy.Module.
# 2. We need its parameters to be added via torch.nn.Module instances.
# 3. Those instances must be instantiated via torch.nn.Module subclasses imported from torch.
# Solution: store the Q-table tensor as the weights in a Linear module and extract/reshape on the client after downloading the model.
# The bias represents the epsilon value
class QLearningAgent(sy.Module):
    def __init__(self, torch_ref: Any, dims: tuple[int, ...], initial_epsilon=1.0, ) -> None:
        super().__init__(torch_ref=torch_ref)
        n_weights = prod(dims)
        self.network = torch_ref.nn.Linear(n_weights, 1, bias=True)
        for p in self.parameters():
            p.requires_grad = False
        torch_ref.nn.init.zeros_(self.network.weight)
        self.network.bias.data = torch_ref.tensor([initial_epsilon], requires_grad=False).data

In [4]:
local_agent = QLearningAgent(T, CARTPOLE_DIMS)

## Step 3: Define the training and averaging plans

In [5]:
# Hack: we need a client plan so PyGrid will allow our client to download the model
# We can't use a training plan right now because the Blackjack env isn't supported yet
@make_plan
def nothing_plan():
    pass

@make_plan
def averaging_plan(
    # Average of diffs, not parameters
    current_average=List(local_agent.parameters()),
    next_diff=List(local_agent.parameters()),
    num=Int(0),
):
    return [
        (current_param * num + diff_param) / (num + 1)
        for current_param, diff_param in zip(current_average, next_diff)
    ]

## Step 4: Host in PyGrid

In [6]:
grid = ModelCentricFLClient(address=GRID_ADDRESS, secure=False)
grid.connect()

In [7]:
response = grid.host_federated_training(
    model=local_agent,
    client_plans={"nothing_plan": nothing_plan},
    client_protocols={},
    server_averaging_plan=averaging_plan,
    client_config=CLIENT_CONFIG,
    server_config=SERVER_CONFIG,
)
print(f"Host response: {response}")

Host response: {'type': 'model-centric/host-training', 'data': {'status': 'success'}}


<div class="alert alert-info">
Now run the Data Owner notebook to train the model.
</div>